In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import datetime as dt

In [3]:
base_path = 'data/I88N-processed/'

# Initialization

## Sample dates, split train/test dataset

In [4]:
def date_to_day(date):
    # date: a date string in the format of "yyyy-mm-dd"
    # return: an int w/t Monday being 0 and Sunday being 6.
    if date.find('-') != -1:
        y, m, d = date.split('-')
    else:
        m, d, y = date.split('/')
    return dt.datetime(int(y), int(m), int(d)).weekday()

In [5]:
dates = pd.read_csv(base_path + 'available_dates.csv')
dates = np.array(dates['0'].values.tolist())
dates = np.array(list(map(lambda x: x.split('-')[1] + '/' + x.split('-')[2] + '/' + x.split('-')[0], dates)))

We want to sample dates in June.

In [6]:
dates = dates[(dates > '05/31/2017') & (dates < '07/01/2017')]

In [7]:
dates_weekend = [date for date in dates if date_to_day(date) >= 5]
dates_weekday = list(set(dates).difference(set(dates_weekend)))

In [8]:
dates_train = dates_weekday[:14] + dates_weekend[:3]
dates_test = dates_weekday[14:] + dates_weekend[3:]
dates_train.sort()
dates_test.sort()

In [9]:
len(dates), len(dates_train), len(dates_test), dates_train[0:3], dates_test[0:3]

(29,
 17,
 12,
 ['06/02/2017', '06/03/2017', '06/04/2017'],
 ['06/01/2017', '06/07/2017', '06/08/2017'])

## Loading severity data

In [10]:
severity_data = pd.read_csv(base_path + 'severity_params.csv')
severity_data.rename(columns={'Unnamed: 0':'datetime'}, inplace=True)

In [11]:
sev_datetimes = severity_data['datetime'].values

In [12]:
sev_dates = []
sev_times = []
for x in sev_datetimes:
    d, t = x.split(' ')
    sev_dates.append(d)
    sev_times.append(t)

In [13]:
severity_data['Date'] = sev_dates
severity_data['Time'] = sev_times

In [14]:
severity_data = severity_data.loc[~severity_data['Date'].isin(['2017-06-15'])]

In [15]:
lambda_max = severity_data['LambdaMax'].values
sigma = severity_data['Sigma'].values
tau = severity_data['Tau'].values
impact = severity_data['Impact'].values

In [16]:
severity_data.head(3)

,datetime,ID,LambdaMax,Sigma,Tau,Impact,Incident,Date,Time
0,2017-06-01 00:00:00,408907,NaN,NaN,NaN,0.021267,0.0,2017-06-01,00:00:00
1,2017-06-01 00:05:00,408907,NaN,NaN,NaN,0.017058,0.0,2017-06-01,00:05:00
2,2017-06-01 00:10:00,408907,NaN,NaN,0.0,0.015338,0.0,2017-06-01,00:10:00


In [17]:
lambda_max = [0 if np.isnan(x) else x for x in lambda_max]
sigma = [0 if np.isnan(x) else x for x in sigma]
tau = [0 if np.isnan(x) else x for x in tau]

In [18]:
severity_data['LambdaMax'] = lambda_max
severity_data['Sigma'] = sigma
severity_data['Tau'] = tau

## Loading speed, flow, occupancy, and stations

In [19]:
raw = pd.read_csv(base_path + 'concat_no_holes/concat.csv')

In [20]:
# select raw that is sampled
raw_all = raw.loc[raw['Date'].isin(dates)]

In [21]:
raw_all['LambdaMax'] = lambda_max
raw_all['Sigma'] = sigma
raw_all['Tau'] = tau
raw_all['Impact'] = impact

/home/hylei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/hylei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/hylei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [22]:
raw_test = raw_all.loc[raw['Date'].isin(dates_test)]
stations = np.array(raw_all['Station ID'].unique().tolist())

In [23]:
raw_all.head(3)

,Station ID,datetime,Occupancy,Flow,Speed,Date,Time,idx,LambdaMax,Sigma,Tau,Impact
8640,408907,2017-06-01 00:00:00,0.5,22.0,69.5,06/01/2017,00:00,42322,0.0,0.0,0.0,0.021267
8641,408907,2017-06-01 00:05:00,0.5,22.0,69.1,06/01/2017,00:05,42323,0.0,0.0,0.0,0.017058
8642,408907,2017-06-01 00:10:00,0.5,23.0,68.9,06/01/2017,00:10,42324,0.0,0.0,0.0,0.015338


In [24]:
# special construction of raw_train, because the dates are sampled with replacement
raw_train = pd.DataFrame()
duplicate_id = 0
for i in range(0, len(dates_train)):
    if i > 0:
        if dates_train[i] == dates_train[i-1]:
            duplicate_id += 1
        else:
            duplicate_id = 0
    df_date = raw_all.loc[raw_all['Date'] == dates_train[i]]
    df_date = df_date.assign(duplicateIdx=duplicate_id)
    raw_train = raw_train.append(df_date)
sorterIdx = dict( zip(stations, range(len(stations))) )
raw_train['stationSorterIdx'] = raw_train['Station ID'].map(sorterIdx)
raw_train = raw_train.sort_values(['stationSorterIdx', 'duplicateIdx', 'datetime'], ascending=[True, True, True])
raw_train.drop(['duplicateIdx', 'stationSorterIdx', 'idx'], axis=1, inplace=True)

In [25]:
len(raw_train.index)

499392

In [26]:
var_names = ['Speed', 'Flow', 'Occupancy']

### Construct road segments

In [27]:
road_segments = list()
for i in range(len(stations) - 1):
    road_segments.append(tuple([stations[i], stations[i+1]]))

## Loading incidents

In [28]:
raw_incidents = pd.read_csv(base_path + 'valid_incidents.csv')

In [29]:
raw_incidents_all = raw_incidents.loc[raw_incidents['Date'].isin(dates)]
raw_incidents_train = raw_incidents_all.loc[raw_incidents_all['Date'].isin(dates_train)]
raw_incidents_test = raw_incidents_all.loc[raw_incidents_all['Date'].isin(dates_test)]

In [30]:
svm_pos_timestamps = pd.read_csv(base_path + 'svm_pos_instances.csv')

In [31]:
svm_pos_timestamps_train = svm_pos_timestamps.loc[svm_pos_timestamps['Date'].isin(dates_train)]
svm_pos_timestamps_test = svm_pos_timestamps.loc[svm_pos_timestamps['Date'].isin(dates_test)]

In [32]:
svm_pos_timestamps.head(3)

,Upstream,Downstream,Date,Time
0,408907,400951,01/22/2017,20:30
1,408907,400951,01/22/2017,20:35
2,408907,400951,01/22/2017,20:40


In [33]:
svm_incident_dates_train = svm_pos_timestamps_train['Date'].unique().tolist()
svm_normal_dates_train = list(set(dates_train).difference(svm_incident_dates_train))

In [34]:
len(svm_incident_dates_train), len(svm_normal_dates_train)

(17, 0)

## Progress message formatting

In [35]:
def fraction_msg(present, total):
    return '[{}/{}]'.format(present, total)

# Train: TSA-DES forecasting

In [36]:
def DES_rmse(alpha, var_series):
    len_series = len(var_series)
    
    beta = round(1. - alpha, 3)

    sse = 0.
    s1 = np.mean(var_series[:10])
    s2 = s1
    
    for i in range(11, len_series - 1):
        s1 = alpha * var_series[i] + beta * s1
        s2 = alpha * s1 + beta * s2
        y_next = 2 * s1 - s2 + alpha / beta * (s1 - s2)
        sse += (var_series[i+1] - y_next) ** 2
    
    return np.sqrt( sse / (len_series - 12) )

## Tune best alphas for each station

In [37]:
import multiprocessing as mp

In [38]:
# input: stations, raw training data (includ. incidents), rule to update alphas
# output: a dictionary containing stations, and stations' best alphas
def compute_best_alphas(stations, raw_train, raw_incidents_train, dates_train, num_grids, DES_rmse, fraction_msg):
    best_alphas = {
    'Station ID': [],
    'Speed': [],
    'Flow': [],
    'Occupancy': []
    }
    pid = mp.current_process().pid
    for i, station in enumerate(stations):
        best_alphas['Station ID'].append(station)

        # update current training station dataframe, the training data is normal day's data
        abnormal_dates_station = raw_incidents_train.loc[(raw_incidents_train['Upstream'] == station) | (raw_incidents_train['Downstream'] == str(station))]['Date'].unique()
        normal_dates_train = np.array(list(set(dates_train).difference(set(abnormal_dates_station))))
        df_train_station = raw_train.loc[(raw_train['Station ID'] == station) & (raw_train['Date'].isin(normal_dates_train))]

        print("{} {} Tuning alphas for station {}...".format(pid, fraction_msg(i+1, len(stations)), station))
        for var_name in var_names:
            # print("    " + var_name + "...")
            var_series = df_train_station[var_name].values
            len_series = len(var_series)

            # setting up alphas
            alphas = np.arange(num_grids) * 1. / num_grids

            # save the historical best alpha by rmse
            best_rmse = float("inf")
            best_alpha = 0.

            # for each alpha, perform exponential smoothing, and compute RMSE
            for alpha in alphas:
                rmse = DES_rmse(alpha, var_series)

                # compare, and decide whether to update best alpha
                if rmse < best_rmse:
                    best_rmse = rmse
                    best_alpha = alpha

            # finally, save the best alpha for the variable at this station
            best_alphas[var_name].append(best_alpha)

        # print trained alphas for each station
        # print(best_alphas['Station ID'][i], best_alphas['Speed'][i], best_alphas['Flow'][i], best_alphas['Occupancy'][i])
    print("Process {} has finished alpha tuning.".format(pid))
    return best_alphas

In [39]:
pool = mp.Pool(processes=8)
num_grids = 100
results = [pool.apply_async(compute_best_alphas, args=(stations[13 * i: 13 * i + 13], raw_train, raw_incidents_train, dates_train, num_grids, DES_rmse, fraction_msg)) for i in range(0, 8)]
pool.close()
pool.join()

29907 [1/13] Tuning alphas for station 408907...
29908 [1/13] Tuning alphas for station 400088...
29909 [1/13] Tuning alphas for station 408755...
29910 [1/13] Tuning alphas for station 400137...
29911 [1/13] Tuning alphas for station 400611...
29912 [1/13] Tuning alphas for station 400275...
29913 [1/13] Tuning alphas for station 400333...
29914 [1/11] Tuning alphas for station 400980...
29908 [2/13] Tuning alphas for station 402288...
29907 [2/13] Tuning alphas for station 400951...
29909 [2/13] Tuning alphas for station 402802...
29910 [2/13] Tuning alphas for station 400716...
29912 [2/13] Tuning alphas for station 400939...
29911 [2/13] Tuning alphas for station 400928...
29913 [2/13] Tuning alphas for station 410363...
29914 [2/11] Tuning alphas for station 401333...
29908 [3/13] Tuning alphas for station 413026...
29907 [3/13] Tuning alphas for station 400057...
29909 [3/13] Tuning alphas for station 408756...
29910 [3/13] Tuning alphas for station 401545...
29912 [3/13] Tuning 

In [40]:
exec_results = []
for proc in results:
    exec_results.append(proc.get())

In [41]:
best_alphas = {
    'Station ID': [],
    'Speed': [],
    'Flow': [],
    'Occupancy': []
}
for dict_best_alphas in exec_results:
    for key in best_alphas.keys():
        best_alphas[key].extend(dict_best_alphas[key])

In [42]:
best_alphas

{'Station ID': [408907,
  400951,
  400057,
  400147,
  400343,
  401560,
  400045,
  400122,
  401541,
  402281,
  402283,
  402285,
  402286,
  400088,
  402288,
  413026,
  401464,
  401489,
  401538,
  402290,
  402292,
  401643,
  402800,
  402828,
  407219,
  402789,
  408755,
  402802,
  408756,
  400189,
  400309,
  400417,
  400249,
  401639,
  400662,
  400141,
  400761,
  400490,
  401888,
  400137,
  400716,
  401545,
  401011,
  400674,
  400539,
  400534,
  401062,
  401529,
  401613,
  400536,
  400488,
  401561,
  400611,
  400928,
  400284,
  400041,
  408133,
  408135,
  417665,
  412637,
  417666,
  408134,
  400685,
  401003,
  400898,
  400275,
  400939,
  400180,
  400529,
  400990,
  400515,
  400252,
  400788,
  401517,
  401871,
  400574,
  401629,
  400422,
  400333,
  410363,
  400360,
  400955,
  400495,
  400608,
  400949,
  400678,
  400341,
  400607,
  400094,
  400682,
  408138,
  400980,
  401333,
  404746,
  401142,
  400218,
  400983,
  400765,
  4008

In [43]:
best_alphas_df = pd.DataFrame(best_alphas)
best_alphas_df.to_csv(base_path + 'smaller_sample/best_alphas.csv', index=False)

## Using the tuned alphas to predict training traffic variables

In [44]:
# initialization
# initialize prediction dictionary
pred_dict_train = dict()
for var_name in var_names:
    pred_dict_train[var_name] = []

for i, station in enumerate(stations):
    print("{} Start time series prediction (DES) at station {}...".format(fraction_msg(i+1, len(stations)), station))
    df_train_station = raw_train.loc[raw_train["Station ID"] == station]
    
    # formulate predictions of speed, flow and occupancy for the station
    for var_name in var_names:
        print("    {}...".format(var_name))
        var_series = df_train_station[var_name].values
        len_series = len(var_series)
        # initialize s1, s2, and y
        s1 = np.mean(var_series[:10])
        s2 = s1
        y = [0.] * len_series
        # get the best alpha
        var_best_alpha = best_alphas_df.loc[best_alphas_df["Station ID"] == station][var_name].values[0]
        beta = 1. - var_best_alpha

        for t in range(11, len_series - 1):
            s1 = var_best_alpha * var_series[t] + beta * s1
            s2 = var_best_alpha * s1 + beta * s2
            y[t+1] = round(2 * s1 - s2 + var_best_alpha / beta * (s1 - s2), 2)

        # save the predictions to a dictionary
        pred_dict_train[var_name].extend(y)
    print("End prediction at station {}.".format(station))

[1/102] Start time series prediction (DES) at station 408907...
    Speed...
    Flow...
    Occupancy...
End prediction at station 408907.
[2/102] Start time series prediction (DES) at station 400951...
    Speed...
    Flow...
    Occupancy...
End prediction at station 400951.
[3/102] Start time series prediction (DES) at station 400057...
    Speed...
    Flow...
    Occupancy...
End prediction at station 400057.
[4/102] Start time series prediction (DES) at station 400147...
    Speed...
    Flow...
    Occupancy...
End prediction at station 400147.
[5/102] Start time series prediction (DES) at station 400343...
    Speed...
    Flow...
    Occupancy...
End prediction at station 400343.
[6/102] Start time series prediction (DES) at station 401560...
    Speed...
    Flow...
    Occupancy...
End prediction at station 401560.
[7/102] Start time series prediction (DES) at station 400045...
    Speed...
    Flow...
    Occupancy...
End prediction at station 400045.
[8/102] Start time s

End prediction at station 412637.
[61/102] Start time series prediction (DES) at station 417666...
    Speed...
    Flow...
    Occupancy...
End prediction at station 417666.
[62/102] Start time series prediction (DES) at station 408134...
    Speed...
    Flow...
    Occupancy...
End prediction at station 408134.
[63/102] Start time series prediction (DES) at station 400685...
    Speed...
    Flow...
    Occupancy...
End prediction at station 400685.
[64/102] Start time series prediction (DES) at station 401003...
    Speed...
    Flow...
    Occupancy...
End prediction at station 401003.
[65/102] Start time series prediction (DES) at station 400898...
    Speed...
    Flow...
    Occupancy...
End prediction at station 400898.
[66/102] Start time series prediction (DES) at station 400275...
    Speed...
    Flow...
    Occupancy...
End prediction at station 400275.
[67/102] Start time series prediction (DES) at station 400939...
    Speed...
    Flow...
    Occupancy...
End predictio

In [45]:
raw_train = raw_train.assign(Pred_Speed=pred_dict_train['Speed'], Pred_Flow=pred_dict_train['Flow'], Pred_Occupancy=pred_dict_train['Occupancy'])

In [46]:
raw_train['Diff_Speed'] = raw_train['Speed'] - raw_train['Pred_Speed']
raw_train['Diff_Flow'] = raw_train['Flow'] - raw_train['Pred_Flow']
raw_train['Diff_Occupancy'] = raw_train['Occupancy'] - raw_train['Pred_Occupancy']

## Using the tuned alphas to predict testing traffic variables

In [47]:
# initialization
# initialize prediction dictionary
pred_dict_test = dict()
for var_name in var_names:
    pred_dict_test[var_name] = []

for i, station in enumerate(stations):
    print("{} Start time series prediction (DES) at station {}...".format(fraction_msg(i+1, len(stations)), station))
    df_test_station = raw_test.loc[raw_test["Station ID"] == station]
    
    # formulate predictions of speed, flow and occupancy for the station
    for var_name in var_names:
        print("    {}...".format(var_name))
        var_series = df_test_station[var_name].values
        len_series = len(var_series)
        # initialize s1, s2, and y
        s1 = np.mean(var_series[:10])
        s2 = s1
        y = [0.] * len_series
        # get the best alpha
        var_best_alpha = best_alphas_df.loc[best_alphas_df["Station ID"] == station][var_name].values[0]
        beta = 1. - var_best_alpha

        num_batches = int(len_series / 288)
        for j in range(num_batches):
            base_idx = 288 * j
            for t in range(base_idx + 11, base_idx + 287):
                s1 = var_best_alpha * var_series[t] + beta * s1
                s2 = var_best_alpha * s1 + beta * s2
                y[t+1] = round(2 * s1 - s2 + var_best_alpha / beta * (s1 - s2), 2)

        # save the predictions to a dictionary
        pred_dict_test[var_name].extend(y)
    print("Finished forecasting at station {}.".format(station))
print("Finished forecasting for the test dataset.")

[1/102] Start time series prediction (DES) at station 408907...
    Speed...
    Flow...
    Occupancy...
Finished forecasting at station 408907.
[2/102] Start time series prediction (DES) at station 400951...
    Speed...
    Flow...
    Occupancy...
Finished forecasting at station 400951.
[3/102] Start time series prediction (DES) at station 400057...
    Speed...
    Flow...
    Occupancy...
Finished forecasting at station 400057.
[4/102] Start time series prediction (DES) at station 400147...
    Speed...
    Flow...
    Occupancy...
Finished forecasting at station 400147.
[5/102] Start time series prediction (DES) at station 400343...
    Speed...
    Flow...
    Occupancy...
Finished forecasting at station 400343.
[6/102] Start time series prediction (DES) at station 401560...
    Speed...
    Flow...
    Occupancy...
Finished forecasting at station 401560.
[7/102] Start time series prediction (DES) at station 400045...
    Speed...
    Flow...
    Occupancy...
Finished forecasti

    Occupancy...
Finished forecasting at station 408133.
[58/102] Start time series prediction (DES) at station 408135...
    Speed...
    Flow...
    Occupancy...
Finished forecasting at station 408135.
[59/102] Start time series prediction (DES) at station 417665...
    Speed...
    Flow...
    Occupancy...
Finished forecasting at station 417665.
[60/102] Start time series prediction (DES) at station 412637...
    Speed...
    Flow...
    Occupancy...
Finished forecasting at station 412637.
[61/102] Start time series prediction (DES) at station 417666...
    Speed...
    Flow...
    Occupancy...
Finished forecasting at station 417666.
[62/102] Start time series prediction (DES) at station 408134...
    Speed...
    Flow...
    Occupancy...
Finished forecasting at station 408134.
[63/102] Start time series prediction (DES) at station 400685...
    Speed...
    Flow...
    Occupancy...
Finished forecasting at station 400685.
[64/102] Start time series prediction (DES) at station 401003

In [48]:
raw_test = raw_test.assign(Pred_Speed=pred_dict_test['Speed'], Pred_Flow=pred_dict_test['Flow'], Pred_Occupancy=pred_dict_test['Occupancy'])

In [49]:
raw_test['Diff_Speed'] = raw_test['Speed'] - raw_test['Pred_Speed']
raw_test['Diff_Flow'] = raw_test['Flow'] - raw_test['Pred_Flow']
raw_test['Diff_Occupancy'] = raw_test['Occupancy'] - raw_test['Pred_Occupancy']

In [50]:
raw_test.tail(3)

,Station ID,datetime,Occupancy,Flow,Speed,Date,Time,idx,LambdaMax,Sigma,Tau,Impact,Pred_Speed,Pred_Flow,Pred_Occupancy,Diff_Speed,Diff_Flow,Diff_Occupancy
6532125,401471,2017-06-28 23:45:00,0.7,23.0,67.3,06/28/2017,23:45,3201453,0.038882,0.0,0.0,0.064840,67.22,23.99,0.69,0.08,-0.99,0.01
6532126,401471,2017-06-28 23:50:00,0.6,23.0,67.2,06/28/2017,23:50,3201454,0.038882,0.0,0.0,0.064408,67.29,21.89,0.67,-0.09,1.11,-0.07
6532127,401471,2017-06-28 23:55:00,0.6,19.0,67.1,06/28/2017,23:55,3201455,0.038882,0.0,0.0,0.063334,67.24,21.68,0.58,-0.14,-2.68,0.02


### Compute z-scores

In [51]:
mean_sd_dict = dict()
pred_var_names = []
diff_var_names = []
for var in var_names:
    mean_sd_dict[var + "_mean"] = []
    mean_sd_dict[var + "_sd"] = []
    pred_var_names.append("Pred_" + var)
    diff_var_names.append("Diff_" + var)
for var in pred_var_names:
    mean_sd_dict[var + "_mean"] = []
    mean_sd_dict[var + "_sd"] = []
for var in diff_var_names:
    mean_sd_dict[var + "_mean"] = []
    mean_sd_dict[var + "_sd"] = []
mean_sd_dict['station'] = stations

for i, s in enumerate(stations):
    s_df = raw_train[raw_train['Station ID'] == s]
    for j, var in enumerate(var_names):
        var_values = s_df[var].values
        mean_sd_dict[var + "_mean"].append(np.mean(var_values))
        mean_sd_dict[var + "_sd"].append(np.std(var_values))
    for j, var in enumerate(pred_var_names):
        var_values = s_df[var].values
        mean_sd_dict[var + "_mean"].append(np.mean(var_values))
        mean_sd_dict[var + "_sd"].append(np.std(var_values))
    for j, var in enumerate(diff_var_names):
        var_values = s_df[var].values
        mean_sd_dict[var + "_mean"].append(np.mean(var_values))
        mean_sd_dict[var + "_sd"].append(np.std(var_values))

In [52]:
stations_train = raw_train['Station ID'].values
station_idx_dict = dict()
for i, s in enumerate(stations):
    station_idx_dict[s] = i

In [53]:
columns = ['Speed', 'Flow', 'Occupancy', 
           'Pred_Speed', 'Pred_Flow', 'Pred_Occupancy', 
           'Diff_Speed', 'Diff_Flow', 'Diff_Occupancy']

In [54]:
z_dict = dict()
for col in columns:
    z_scores = []
    col_train = raw_train[col].values
    for i, val in enumerate(col_train):
        sid = stations_train[i]
        sidx = station_idx_dict[sid]
        z_score = (val - mean_sd_dict[col+'_mean'][sidx]) / mean_sd_dict[col+'_sd'][sidx]
        z_scores.append(z_score)
    z_dict[col] = z_scores
    print("Finished z-score computation: {}.".format(col))

Finished z-score computation: Speed.
Finished z-score computation: Flow.
Finished z-score computation: Occupancy.
Finished z-score computation: Pred_Speed.
Finished z-score computation: Pred_Flow.
Finished z-score computation: Pred_Occupancy.
Finished z-score computation: Diff_Speed.
Finished z-score computation: Diff_Flow.
Finished z-score computation: Diff_Occupancy.


In [55]:
for key in z_dict.keys():
    raw_train[key] = z_dict[key]

In [56]:
stations_test = raw_test['Station ID'].values
z_dict_test = dict()
for col in columns:
    z_scores = []
    col_test = raw_test[col].values
    for i, val in enumerate(col_test):
        sid = stations_test[i]
        sidx = station_idx_dict[sid]
        z_score = (val - mean_sd_dict[col+'_mean'][sidx]) / mean_sd_dict[col+'_sd'][sidx]
        z_scores.append(z_score)
    z_dict_test[col] = z_scores
    print("Finished z-score computation: {}.".format(col))

Finished z-score computation: Speed.
Finished z-score computation: Flow.
Finished z-score computation: Occupancy.
Finished z-score computation: Pred_Speed.
Finished z-score computation: Pred_Flow.
Finished z-score computation: Pred_Occupancy.
Finished z-score computation: Diff_Speed.
Finished z-score computation: Diff_Flow.
Finished z-score computation: Diff_Occupancy.


In [57]:
for key in z_dict.keys():
    raw_test[key] = z_dict_test[key]

## Weekday/weekend Indicator

In [58]:
raw_train_dates = raw_train['Date'].values

In [59]:
def is_weekday(weekday):
    if weekday - 5 >= 0:
        return 0
    else:
        return 1

In [60]:
raw_train_weekday = [is_weekday(date_to_day(date)) for date in raw_train_dates]

In [61]:
raw_train['weekday'] = raw_train_weekday

In [62]:
raw_train.head(3)

,Station ID,datetime,Occupancy,Flow,Speed,Date,Time,LambdaMax,Sigma,Tau,Impact,Pred_Speed,Pred_Flow,Pred_Occupancy,Diff_Speed,Diff_Flow,Diff_Occupancy,weekday
8928,408907,2017-06-02 00:00:00,-0.953162,-1.373286,0.586367,06/02/2017,00:00,0.0,0.0,0.0,0.016859,-10.800253,-1.611214,-1.072184,18.179947,1.214776,0.446144,1
8929,408907,2017-06-02 00:05:00,-0.981987,-1.341938,0.565064,06/02/2017,00:05,0.0,0.0,0.0,0.017058,-10.800253,-1.611214,-1.072184,18.153611,1.360780,0.382282,1
8930,408907,2017-06-02 00:10:00,-1.010811,-1.415085,0.586367,06/02/2017,00:10,0.0,0.0,0.0,0.019758,-10.800253,-1.611214,-1.072184,18.179947,1.020103,0.318421,1


# Train: SVM

Note that we need to rescale train and test dataset with the same factors.

## Train: feature vectors

### Train: feature vectors - negative

In [63]:
neg_times = raw_train['Time'].unique().tolist()[14:]

In [64]:
neg_sample_dates = dates_train

In [65]:
svm_incidents_sample = svm_pos_timestamps_train.loc[svm_pos_timestamps_train['Date'].isin(neg_sample_dates)]

In [66]:
X_neg_train = []
num_segments = len(road_segments)
count_date = 0
for neg_sample_date in neg_sample_dates:
    count_date += 1
    print("{} Negative feature vectors at date {}:".format(fraction_msg(count_date, len(neg_sample_dates)), neg_sample_date))
    
    for i, seg in enumerate(road_segments):
        B, E = seg
        df_neg_train_BE = raw_train.loc[((raw_train["Station ID"] == B) | (raw_train["Station ID"] == E)) & (raw_train["Date"] == neg_sample_date)]
        svm_incidents_sample_BE = svm_incidents_sample.loc[svm_incidents_sample['Upstream'] == B]
        sample_neg_times = np.random.choice(neg_times, 24)
        
        if (i+1) % 20 == 0:
            print("    {} Start constructing feature vectors for road segment s_{},{}...".format(fraction_msg(i+1, num_segments), B, E))
            print("        Total number of vectors: {}".format(len(sample_neg_times)))
        
        for neg_t in sample_neg_times:
            # check if current time is incident time
            if len(svm_incidents_sample_BE.loc[svm_incidents_sample_BE['Time'] == neg_t].index) != 0:
                continue

            feature_t = []
            neg_dt_timestamp = pd.Timestamp(neg_sample_date + ' ' + neg_t + ':00')

            B_lags = []
            for j in range(5):
                B_lags.append(neg_dt_timestamp - dt.timedelta(minutes=j*5))
            B_lags = list(map(lambda x: x.strftime('%H:%M') , B_lags))
            E_lags = B_lags[0:3]

            # upstream features
            for t_lag in B_lags:
                df_dt_lag = df_neg_train_BE.loc[(df_neg_train_BE["Station ID"] == B) & (df_neg_train_BE["Time"] == t_lag)]

                speed_B_t = df_dt_lag["Speed"].values[0]
                flow_B_t = df_dt_lag["Flow"].values[0]
                occ_B_t = df_dt_lag["Occupancy"].values[0]

                speed_pred_B_t = df_dt_lag["Pred_Speed"].values[0]
                flow_pred_B_t = df_dt_lag["Pred_Flow"].values[0]
                occ_pred_B_t = df_dt_lag["Pred_Occupancy"].values[0]
 
                speed_diff_B_t = df_dt_lag["Diff_Speed"].values[0]
                flow_diff_B_t = df_dt_lag["Diff_Flow"].values[0]
                occ_diff_B_t = df_dt_lag["Diff_Occupancy"].values[0]
            
                lambda_max_B_t = df_dt_lag["LambdaMax"].values[0]
                sigma_B_t = df_dt_lag["Sigma"].values[0]
                tau_B_t = df_dt_lag["Tau"].values[0]
                impact_B_t = df_dt_lag["Impact"].values[0]

                weekday_B_t = df_dt_lag["weekday"].values[0]
                
                feature_t.extend([speed_B_t, flow_B_t, occ_B_t, 
                                  speed_pred_B_t, flow_pred_B_t, occ_pred_B_t, 
                                  speed_diff_B_t, flow_diff_B_t, occ_diff_B_t, 
                                  lambda_max_B_t, sigma_B_t, tau_B_t, impact_B_t,
                                  weekday_B_t])

            # downstream features
            for t_lag in E_lags:
                df_dt_lag = df_neg_train_BE.loc[(df_neg_train_BE["Station ID"] == E) & (df_neg_train_BE["Time"] == t_lag)]

                speed_E_t = df_dt_lag["Speed"].values[0]
                flow_E_t = df_dt_lag["Flow"].values[0]
                occ_E_t = df_dt_lag["Occupancy"].values[0]

                speed_pred_E_t = df_dt_lag["Pred_Speed"].values[0]
                flow_pred_E_t = df_dt_lag["Pred_Flow"].values[0]
                occ_pred_E_t = df_dt_lag["Pred_Occupancy"].values[0]

                speed_diff_E_t = df_dt_lag["Diff_Speed"].values[0]
                flow_diff_E_t = df_dt_lag["Diff_Flow"].values[0]
                occ_diff_E_t = df_dt_lag["Diff_Occupancy"].values[0]
                
                lambda_max_E_t = df_dt_lag["LambdaMax"].values[0]
                sigma_E_t = df_dt_lag["Sigma"].values[0]
                tau_E_t = df_dt_lag["Tau"].values[0]
                impact_E_t = df_dt_lag["Impact"].values[0]
                
                weekday_E_t = df_dt_lag["weekday"].values[0]
                
                feature_t.extend([speed_E_t, flow_E_t, occ_E_t, 
                                  speed_pred_E_t, flow_pred_E_t, occ_pred_E_t, 
                                  speed_diff_E_t, flow_diff_E_t, occ_diff_E_t, 
                                  lambda_max_E_t, sigma_E_t, tau_E_t, impact_E_t, 
                                  weekday_E_t])
            
            X_neg_train.append(feature_t)

        if (i+1) % 20 == 0:
            print("        Feature vector at date and time {} {} is done.".format(neg_sample_date, neg_t))
            print("    ...Completed construction of feature vectors for road segment s_{},{}.".format(B, E))

[1/17] Negative feature vectors at date 06/02/2017:
    [20/101] Start constructing feature vectors for road segment s_402290,402292...
        Total number of vectors: 24
        Feature vector at date and time 06/02/2017 15:50 is done.
    ...Completed construction of feature vectors for road segment s_402290,402292.
    [40/101] Start constructing feature vectors for road segment s_400137,400716...
        Total number of vectors: 24
        Feature vector at date and time 06/02/2017 09:20 is done.
    ...Completed construction of feature vectors for road segment s_400137,400716.
    [60/101] Start constructing feature vectors for road segment s_412637,417666...
        Total number of vectors: 24
        Feature vector at date and time 06/02/2017 23:40 is done.
    ...Completed construction of feature vectors for road segment s_412637,417666.
    [80/101] Start constructing feature vectors for road segment s_410363,400360...
        Total number of vectors: 24
        Feature vecto

        Feature vector at date and time 06/09/2017 10:20 is done.
    ...Completed construction of feature vectors for road segment s_400923,401143.
[7/17] Negative feature vectors at date 06/10/2017:
    [20/101] Start constructing feature vectors for road segment s_402290,402292...
        Total number of vectors: 24
        Feature vector at date and time 06/10/2017 12:45 is done.
    ...Completed construction of feature vectors for road segment s_402290,402292.
    [40/101] Start constructing feature vectors for road segment s_400137,400716...
        Total number of vectors: 24
        Feature vector at date and time 06/10/2017 21:45 is done.
    ...Completed construction of feature vectors for road segment s_400137,400716.
    [60/101] Start constructing feature vectors for road segment s_412637,417666...
        Total number of vectors: 24
        Feature vector at date and time 06/10/2017 17:55 is done.
    ...Completed construction of feature vectors for road segment s_412637,

    [100/101] Start constructing feature vectors for road segment s_400923,401143...
        Total number of vectors: 24
        Feature vector at date and time 06/20/2017 18:15 is done.
    ...Completed construction of feature vectors for road segment s_400923,401143.
[13/17] Negative feature vectors at date 06/22/2017:
    [20/101] Start constructing feature vectors for road segment s_402290,402292...
        Total number of vectors: 24
        Feature vector at date and time 06/22/2017 12:30 is done.
    ...Completed construction of feature vectors for road segment s_402290,402292.
    [40/101] Start constructing feature vectors for road segment s_400137,400716...
        Total number of vectors: 24
        Feature vector at date and time 06/22/2017 01:30 is done.
    ...Completed construction of feature vectors for road segment s_400137,400716.
    [60/101] Start constructing feature vectors for road segment s_412637,417666...
        Total number of vectors: 24
        Feature vec

In [67]:
len(X_neg_train)

38365

In [68]:
y_neg_train = [-1] * len(X_neg_train)

### Train: feature vectors - positive

In [69]:
working_time = raw_train['Time'].unique().tolist()[14:]

In [70]:
svm_pos_timestamps_train = svm_pos_timestamps_train.loc[svm_pos_timestamps_train['Time'].isin(working_time)]

In [71]:
X_pos_train = []
for i, seg in enumerate(road_segments):
    B, E = seg
    print("{} Start constructing positive feature vectors for road segment s_{},{}... ".format(fraction_msg(i+1, len(road_segments)), B, E))
    progress_count = 0
    
    # construct segment-specific pos_times
    pos_times = []
    df_seg_incidents = svm_pos_timestamps_train.loc[svm_pos_timestamps_train["Upstream"] == B]
    seg_dates = df_seg_incidents['Date'].values.tolist()
    seg_times = df_seg_incidents['Time'].values.tolist()
    num_seg_instances = len(seg_dates)
    for i in range(num_seg_instances):
        pos_times.append(tuple([seg_dates[i], seg_times[i]]))
    
    # select the relevant training data for segment B, E 
    df_train_BE = raw_train.loc[(raw_train["Station ID"] == B) | (raw_train["Station ID"] == E)]
    
    
    print("    Total number of vectors: {}".format(num_seg_instances))
    for pos_dt in pos_times:
        pos_d, pos_t = pos_dt
        feature_t = []
        pos_dt_timestamp = pd.Timestamp(pos_d + ' ' + pos_t + ':00')

        # upstream and downstream time lags
        B_lags = []
        for j in range(5):
            B_lags.append(pos_dt_timestamp - dt.timedelta(minutes=j*5))
        B_lags = list(map(lambda x: (x.strftime('%m/%d/%Y'), x.strftime('%H:%M')) , B_lags))
        E_lags = B_lags[0:3]

        # upstream features
        for dt_lag in B_lags:
            d_lag, t_lag = dt_lag
            df_dt_lag = df_train_BE.loc[(df_train_BE["Station ID"] == B) & (df_train_BE["Date"] == d_lag) & (df_train_BE["Time"] == t_lag)]
            if df_dt_lag.empty:
                print(d_lag, t_lag)
            
            speed_B_t = df_dt_lag["Speed"].values[0]
            flow_B_t = df_dt_lag["Flow"].values[0]
            occ_B_t = df_dt_lag["Occupancy"].values[0]

            speed_pred_B_t = df_dt_lag["Pred_Speed"].values[0]
            flow_pred_B_t = df_dt_lag["Pred_Flow"].values[0]
            occ_pred_B_t = df_dt_lag["Pred_Occupancy"].values[0]
            
            speed_diff_B_t = df_dt_lag["Diff_Speed"].values[0]
            flow_diff_B_t = df_dt_lag["Diff_Flow"].values[0]
            occ_diff_B_t = df_dt_lag["Diff_Occupancy"].values[0]
            
            lambda_max_B_t = df_dt_lag["LambdaMax"].values[0]
            sigma_B_t = df_dt_lag["Sigma"].values[0]
            tau_B_t = df_dt_lag["Tau"].values[0]
            impact_B_t = df_dt_lag["Impact"].values[0]
            
            weekday_B_t = df_dt_lag["weekday"].values[0]

            feature_t.extend([speed_B_t, flow_B_t, occ_B_t, 
                              speed_pred_B_t, flow_pred_B_t, occ_pred_B_t, 
                              speed_diff_B_t, flow_diff_B_t, occ_diff_B_t, 
                              lambda_max_B_t, sigma_B_t, tau_B_t, impact_B_t,
                              weekday_B_t])

        # downstream features
        for dt_lag in E_lags:
            d_lag, t_lag = dt_lag
            df_dt_lag = df_train_BE.loc[(df_train_BE["Station ID"] == E) & (df_train_BE["Date"] == d_lag) & (df_train_BE["Time"] == t_lag)]

            speed_E_t = df_dt_lag["Speed"].values[0]
            flow_E_t = df_dt_lag["Flow"].values[0]
            occ_E_t = df_dt_lag["Occupancy"].values[0]

            speed_pred_E_t = df_dt_lag["Pred_Speed"].values[0]
            flow_pred_E_t = df_dt_lag["Pred_Flow"].values[0]
            occ_pred_E_t = df_dt_lag["Pred_Occupancy"].values[0]
            
            speed_diff_E_t = df_dt_lag["Diff_Speed"].values[0]
            flow_diff_E_t = df_dt_lag["Diff_Flow"].values[0]
            occ_diff_E_t = df_dt_lag["Diff_Occupancy"].values[0]
            
            lambda_max_E_t = df_dt_lag["LambdaMax"].values[0]
            sigma_E_t = df_dt_lag["Sigma"].values[0]
            tau_E_t = df_dt_lag["Tau"].values[0]
            impact_E_t = df_dt_lag["Impact"].values[0]
            
            weekday_E_t = df_dt_lag["weekday"].values[0]
            
            feature_t.extend([speed_E_t, flow_E_t, occ_E_t, 
                              speed_pred_E_t, flow_pred_E_t, occ_pred_E_t, 
                              speed_diff_E_t, flow_diff_E_t, occ_diff_E_t,  
                              lambda_max_E_t, sigma_E_t, tau_E_t, impact_E_t,
                              weekday_E_t,])
        
        X_pos_train.append(feature_t)
        
        progress_count += 1
        if progress_count % 100 == 0:
            print("    {} Feature vector at date and time {} {} is done.".format(fraction_msg(progress_count, num_seg_instances), pos_d, pos_t))

print("...Completed construction of feature vectors for road segment s_{},{}.".format(B, E))

[1/101] Start constructing positive feature vectors for road segment s_408907,400951... 
    Total number of vectors: 0
[2/101] Start constructing positive feature vectors for road segment s_400951,400057... 
    Total number of vectors: 37
[3/101] Start constructing positive feature vectors for road segment s_400057,400147... 
    Total number of vectors: 42
[4/101] Start constructing positive feature vectors for road segment s_400147,400343... 
    Total number of vectors: 38
[5/101] Start constructing positive feature vectors for road segment s_400343,401560... 
    Total number of vectors: 24
[6/101] Start constructing positive feature vectors for road segment s_401560,400045... 
    Total number of vectors: 14
[7/101] Start constructing positive feature vectors for road segment s_400045,400122... 
    Total number of vectors: 13
[8/101] Start constructing positive feature vectors for road segment s_400122,401541... 
    Total number of vectors: 5
[9/101] Start constructing positiv

[72/101] Start constructing positive feature vectors for road segment s_400252,400788... 
    Total number of vectors: 26
[73/101] Start constructing positive feature vectors for road segment s_400788,401517... 
    Total number of vectors: 56
[74/101] Start constructing positive feature vectors for road segment s_401517,401871... 
    Total number of vectors: 0
[75/101] Start constructing positive feature vectors for road segment s_401871,400574... 
    Total number of vectors: 29
[76/101] Start constructing positive feature vectors for road segment s_400574,401629... 
    Total number of vectors: 20
[77/101] Start constructing positive feature vectors for road segment s_401629,400422... 
    Total number of vectors: 44
[78/101] Start constructing positive feature vectors for road segment s_400422,400333... 
    Total number of vectors: 0
[79/101] Start constructing positive feature vectors for road segment s_400333,410363... 
    Total number of vectors: 18
[80/101] Start constructin

In [72]:
len(X_pos_train)

2065

In [73]:
y_pos_train = [1] * len(X_pos_train)

## Train: Merging feature vectors together

In [74]:
X_neg_train = np.array(X_neg_train)

In [75]:
X_neg_train_balanced = X_neg_train[np.random.choice(len(X_neg_train), len(X_pos_train), replace=False)].tolist()

In [76]:
y_neg_train_balanced = [-1] * len(X_neg_train_balanced)

In [77]:
X_train = X_neg_train_balanced + X_pos_train

In [78]:
y_train = y_neg_train_balanced + y_pos_train

In [79]:
len(X_train), len(y_train)

(4130, 4130)

## Test: feature vectors

In [80]:
raw_test.tail(3)

,Station ID,datetime,Occupancy,Flow,Speed,Date,Time,idx,LambdaMax,Sigma,Tau,Impact,Pred_Speed,Pred_Flow,Pred_Occupancy,Diff_Speed,Diff_Flow,Diff_Occupancy
6532125,401471,2017-06-28 23:45:00,-1.146233,-1.207703,0.851742,06/28/2017,23:45,3201453,0.038882,0.0,0.0,0.064840,0.273968,-1.181695,-1.144224,-0.025454,-0.307671,0.085939
6532126,401471,2017-06-28 23:50:00,-1.213813,-1.207703,0.752468,06/28/2017,23:50,3201454,0.038882,0.0,0.0,0.064408,0.294311,-1.220799,-1.157647,-0.076512,0.319242,-0.697778
6532127,401471,2017-06-28 23:55:00,-1.213813,-1.282610,0.653193,06/28/2017,23:55,3201455,0.038882,0.0,0.0,0.063334,0.279781,-1.224709,-1.218050,-0.091529,-0.812187,0.183904


In [81]:
raw_test_dates = raw_test['Date'].values

In [82]:
raw_test_weekday = [is_weekday(date_to_day(date)) for date in raw_test_dates]

In [83]:
raw_test['weekday'] = raw_test_weekday

In [84]:
feature_names = ['Speed', 'Flow', 'Occupancy', 
                 'Pred_Speed', 'Pred_Flow', 'Pred_Occupancy', 
                 'Diff_Speed', 'Diff_Flow', 'Diff_Occupancy']

In [85]:
feature_names.extend(['LambdaMax', 'Sigma', 'Tau', 'Impact'])

In [86]:
feature_names.extend(['weekday'])

In [87]:
num_features = len(feature_names)
k_B = 4
k_E = 2

In [88]:
X_test = []
y_test = []
for seg_idx, seg in enumerate(road_segments):
    B, E = seg
    print("{} Constructing feature vector for segment s_{},{}...".format(fraction_msg(seg_idx+1, len(road_segments)), B, E))
    df_BE_test = raw_test.loc[((raw_test["Station ID"] == B) | (raw_test["Station ID"] == E))]
    df_incidents_BE_test = svm_pos_timestamps_test.loc[svm_pos_timestamps_test["Upstream"] == B]
    incidents_BE_date = df_incidents_BE_test["Date"].values
    incidents_BE_time = df_incidents_BE_test["Time"].values
    
    incidents_BE_dt = set()
    for i in range(len(incidents_BE_date)):
        incidents_BE_dt.add(incidents_BE_date[i] + ' ' + incidents_BE_time[i])
    
    # change to access by indices, to make program faster
    features_BE_dict = dict()
    features_BE_dict[B] = dict()
    features_BE_dict[E] = dict()
    for feature_name in feature_names:
        features_BE_dict[B][feature_name] = df_BE_test.loc[df_BE_test["Station ID"] == B][feature_name].values.tolist()
        features_BE_dict[E][feature_name] = df_BE_test.loc[df_BE_test["Station ID"] == E][feature_name].values.tolist()
    
    total_count = len(dates_test) * len(neg_times)
    count = 0
    print("    Total number of instances: {}".format(total_count)) 
    
    for i, d in enumerate(dates_test):
        for j, t in enumerate(neg_times):
            # construct vector Z(s_BE, dt)
            feature_BE_t = [0.] * (k_B + k_E + 2) * num_features
            base_idx = i * 288 + 14 + j
            for k, feature_name in enumerate(feature_names):
                # feature_k_B_t: [t-4, t-3, ..., t] -> need to be reversed and made consistent with order of SVM features. Same to E.
                feature_k_B_t = features_BE_dict[B][feature_name][base_idx-k_B:base_idx+1]
                feature_k_E_t = features_BE_dict[E][feature_name][base_idx-k_E:base_idx+1]
                feature_k_B_t.reverse()
                feature_k_E_t.reverse()
                feature_k_BE_t = feature_k_B_t + feature_k_E_t
                feature_BE_t[k:(k_B + k_E + 2)*num_features:num_features] = feature_k_BE_t
            
            X_test.append(feature_BE_t)
            # label data
            if d + ' ' + t in incidents_BE_dt:
                y_test.append(1)
            else:
                y_test.append(-1)
        count += 1
        if count % 50 == 0:
            print("    Progress: {}" + fraction_msg(count * len(neg_times), total_count))
    print("...Finished construction for segment s_{},{}.".format(B, E))

[1/101] Constructing feature vector for segment s_408907,400951...
    Total number of instances: 3288
...Finished construction for segment s_408907,400951.
[2/101] Constructing feature vector for segment s_400951,400057...
    Total number of instances: 3288
...Finished construction for segment s_400951,400057.
[3/101] Constructing feature vector for segment s_400057,400147...
    Total number of instances: 3288
...Finished construction for segment s_400057,400147.
[4/101] Constructing feature vector for segment s_400147,400343...
    Total number of instances: 3288
...Finished construction for segment s_400147,400343.
[5/101] Constructing feature vector for segment s_400343,401560...
    Total number of instances: 3288
...Finished construction for segment s_400343,401560.
[6/101] Constructing feature vector for segment s_401560,400045...
    Total number of instances: 3288
...Finished construction for segment s_401560,400045.
[7/101] Constructing feature vector for segment s_400045,4

...Finished construction for segment s_400928,400284.
[55/101] Constructing feature vector for segment s_400284,400041...
    Total number of instances: 3288
...Finished construction for segment s_400284,400041.
[56/101] Constructing feature vector for segment s_400041,408133...
    Total number of instances: 3288
...Finished construction for segment s_400041,408133.
[57/101] Constructing feature vector for segment s_408133,408135...
    Total number of instances: 3288
...Finished construction for segment s_408133,408135.
[58/101] Constructing feature vector for segment s_408135,417665...
    Total number of instances: 3288
...Finished construction for segment s_408135,417665.
[59/101] Constructing feature vector for segment s_417665,412637...
    Total number of instances: 3288
...Finished construction for segment s_417665,412637.
[60/101] Constructing feature vector for segment s_412637,417666...
    Total number of instances: 3288
...Finished construction for segment s_412637,417666

In [89]:
len(X_test), len(y_test)

(332088, 332088)

In [90]:
from sklearn import preprocessing

In [91]:
X_normalized = X_train + X_test

In [92]:
X_train_normalized = X_normalized[:len(X_train)]
X_test_normalized = X_normalized[len(X_train):]

## SVM training

In [93]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [94]:
param_grid = {
    'C': [10 ** i for i in range(-4, 3)],
    'gamma': [10 ** i for i in range(-4, 4, 2)]
}

In [95]:
svm_grid_search = GridSearchCV(SVC(kernel='rbf'), n_jobs=8, 
                               param_grid=param_grid, cv=5, 
                               scoring='accuracy', verbose=5)

In [96]:
svm_grid_search.fit(X_train_normalized, y_train)

Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   15.8s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:  4.9min
[Parallel(n_jobs=8)]: Done 140 out of 140 | elapsed: 11.4min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=8,
       param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [0.0001, 0.01, 1, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=5)

In [97]:
svm_grid_search.best_params_

{'C': 100, 'gamma': 0.0001}

In [98]:
svm_grid_search.best_score_

0.6217917675544794

In [99]:
len(svm_grid_search.best_estimator_.support_vectors_)

3323

In [100]:
svm_grid_search.best_estimator_

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [101]:
y_train_pred = svm_grid_search.predict(X_train_normalized)

In [102]:
accuracy_score(y_train_pred, y_train)

0.6399515738498789

In [103]:
len(X_test_normalized)

332088

In [104]:
# parallelize prediction
def predict_func(X, clf):
    print("Process {} is predicting ...".format(mp.current_process().pid))
    return clf.predict(X)

In [105]:
pred_pool = mp.Pool(8)
num_instances = int(np.ceil(len(X_test_normalized) / 8))
y_test_pred_jobs = [pred_pool.apply_async(predict_func, args=(X_test_normalized[i*num_instances:(i+1)*num_instances], svm_grid_search)) for i in range(0, 8)]
pred_pool.close()
pred_pool.join()

Process 30545 is predicting ...
Process 30546 is predicting ...
Process 30547 is predicting ...
Process 30548 is predicting ...
Process 30549 is predicting ...
Process 30550 is predicting ...
Process 30551 is predicting ...
Process 30552 is predicting ...


In [106]:
y_test_pred = []
for y_test_pred_job in y_test_pred_jobs:
    y_test_pred.extend(y_test_pred_job.get())

In [107]:
len(X_test_normalized), len(y_test_pred)

(332088, 332088)

In [108]:
accuracy_score(y_test_pred, y_test)

0.8671948399219483

In [109]:
num_dt_segments = int(len(y_test_pred) / (288-14))

## Decision Tree

In [110]:
from sklearn.tree import DecisionTreeClassifier

In [111]:
clf_gini = DecisionTreeClassifier(criterion='gini', random_state=None)

In [112]:
clf_gini.fit(X_train_normalized, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [113]:
y_train_pred_gini = clf_gini.predict(X_train_normalized)

In [114]:
accuracy_score(y_train_pred_gini, y_train)

1.0

In [115]:
y_test_pred_gini = clf_gini.predict(X_test_normalized)

In [116]:
accuracy_score(y_test_pred_gini, y_test)

0.6734118667341187

## KNN

In [117]:
from sklearn.neighbors import KNeighborsClassifier

In [118]:
neigh = KNeighborsClassifier(n_neighbors=10)

In [119]:
neigh.fit(X_train_normalized, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [120]:
y_train_pred_knn = neigh.predict(X_train_normalized)

In [121]:
accuracy_score(y_train_pred_knn, y_train)

0.7300242130750605

In [122]:
y_test_pred_knn = neigh.predict(X_test_normalized)

In [123]:
accuracy_score(y_test_pred_knn, y_test)

0.7431313386813134

## Random Forest

In [124]:
from sklearn.ensemble import RandomForestClassifier

In [125]:
rdm_forest_clf = RandomForestClassifier(n_estimators=90, max_depth=2)

In [126]:
rdm_forest_clf.fit(X_train_normalized, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=90, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [127]:
y_train_pred_rdm_forest = rdm_forest_clf.predict(X_train_normalized)

In [128]:
accuracy_score(y_train_pred_rdm_forest, y_train)

0.6411622276029055

In [129]:
y_test_pred_rdm_forest = rdm_forest_clf.predict(X_test_normalized)

In [130]:
accuracy_score(y_test_pred_rdm_forest, y_test)

0.8152266869022669

## Gradient Boosting

In [131]:
from sklearn.ensemble import GradientBoostingClassifier

In [132]:
params = {'n_estimators': 90}

In [133]:
grad_boost_clf = GradientBoostingClassifier(**params)

In [134]:
grad_boost_clf.fit(X_train_normalized, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=90,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [135]:
y_train_pred_gb = grad_boost_clf.predict(X_train_normalized)

In [136]:
accuracy_score(y_train_pred_gb, y_train)

0.7968523002421307

In [137]:
y_test_pred_gb = grad_boost_clf.predict(X_test_normalized)

In [138]:
accuracy_score(y_test_pred_gb, y_test)

0.766408301414083

## AdaBoost

In [139]:
from sklearn.ensemble import AdaBoostClassifier

In [140]:
ada_param_grid = {
    'base_estimator': [DecisionTreeClassifier(max_depth=2)],
    'n_estimators': [90],
    'learning_rate': [1]
}

In [141]:
ada_grid_search = GridSearchCV(AdaBoostClassifier(), n_jobs=8, 
                               param_grid=ada_param_grid, cv=3, 
                               scoring='accuracy', verbose=4)

In [142]:
ada_grid_search.fit(X_train_normalized, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   3 | elapsed:   20.4s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   3 out of   3 | elapsed:   20.4s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params=None, iid='warn', n_jobs=8,
       param_grid={'base_estimator': [DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')], 'n_estimators': [90], 'learning_rate': [1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=4)

In [143]:
ada_grid_search.best_params_, ada_grid_search.best_score_

({'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, presort=False, random_state=None,
              splitter='best'), 'learning_rate': 1, 'n_estimators': 90},
 0.5513317191283293)

In [144]:
y_test_pred_ada = ada_grid_search.predict(X_test_normalized)

In [145]:
accuracy_score(y_test, y_test_pred_ada)

0.7022566307725663

## MLP

In [146]:
from sklearn.neural_network import MLPClassifier

In [147]:
mlp_clf = MLPClassifier(learning_rate_init=0.001, max_iter=1000, activation='tanh')
mlp_clf.fit(X_train_normalized, y_train)

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [148]:
y_test_pred_mlp = mlp_clf.predict(X_test_normalized)

In [149]:
accuracy_score(y_test_pred_mlp, y_test)

0.6644654428946545

# Evaluation Metrics

Rush hours: 6:30 AM - 9:00 AM (idx 78 - idx 108), 3:30 PM - 6:30 PM (idx 186 - idx 222).

In [150]:
rush_hours_idx = list(range(78, 109)) + list(range(186, 223))

In [151]:
PT_thresholds = [2 ** i for i in range(5)]

In [152]:
results = ""

## Detection rate (DR)

In [153]:
def DR(y_test, y_test_pred, PT_thresholds, num_dt_segments):
    DRs = []
    for PT_threshold in PT_thresholds:
        num_detected_incidents = 0
        total_num_incidents = 0
        for i in range(num_dt_segments):
            base_idx = i * 274
            max_base_offset = (i + 1) * 274
            start_idx = base_idx
            end_idx = start_idx
            while start_idx < max_base_offset:
                while start_idx < max_base_offset and y_test[start_idx] == -1:
                    start_idx += 1
                if start_idx == max_base_offset:
                    break
                # an incident happens
                # time span of the incident
                end_idx = start_idx
                while end_idx < max_base_offset and y_test[end_idx] == 1:
                    end_idx += 1
                if (end_idx - start_idx >= PT_threshold):
                    total_num_incidents += 1
                    # the incident is detected
                    if 1 in y_test_pred[start_idx:end_idx]:
                        num_detected_incidents += 1

                start_idx = end_idx + 1
        DRs.append(round(num_detected_incidents / total_num_incidents * 100, 2))
        print("PT_{}".format(PT_threshold))
        print("# of detected incidents: {}".format(num_detected_incidents))
        print("Total # of incidents: {}".format(total_num_incidents))
        print("Detection rate: {}".format(DRs[-1]))
    return DRs

### SVM

In [154]:
DRs = DR(y_test, y_test_pred, PT_thresholds[0:3], num_dt_segments)
results += "SVM DR: {}\n".format(DRs[0])

PT_1
# of detected incidents: 58
Total # of incidents: 125
Detection rate: 46.4
PT_2
# of detected incidents: 57
Total # of incidents: 119
Detection rate: 47.9
PT_4
# of detected incidents: 50
Total # of incidents: 105
Detection rate: 47.62


### Decision Tree with Gini Index

In [155]:
DRs = DR(y_test, y_test_pred_gini, PT_thresholds[0:3], num_dt_segments)
results += "DT DR: {}\n".format(DRs[0])

PT_1
# of detected incidents: 102
Total # of incidents: 125
Detection rate: 81.6
PT_2
# of detected incidents: 99
Total # of incidents: 119
Detection rate: 83.19
PT_4
# of detected incidents: 90
Total # of incidents: 105
Detection rate: 85.71


### KNN

In [156]:
DRs = DR(y_test, y_test_pred_knn, PT_thresholds[0:3], num_dt_segments)
results += "KNN DR: {}\n".format(DRs[0])

PT_1
# of detected incidents: 87
Total # of incidents: 125
Detection rate: 69.6
PT_2
# of detected incidents: 86
Total # of incidents: 119
Detection rate: 72.27
PT_4
# of detected incidents: 82
Total # of incidents: 105
Detection rate: 78.1


### Random Forest

In [157]:
DRs = DR(y_test, y_test_pred_rdm_forest, PT_thresholds[0:3], num_dt_segments)
results += "RF DR: {}\n".format(DRs[0])

PT_1
# of detected incidents: 67
Total # of incidents: 125
Detection rate: 53.6
PT_2
# of detected incidents: 63
Total # of incidents: 119
Detection rate: 52.94
PT_4
# of detected incidents: 56
Total # of incidents: 105
Detection rate: 53.33


### Gradient Boosting

In [158]:
DRs = DR(y_test, y_test_pred_gb, PT_thresholds[0:3], num_dt_segments)
results += "GB DR: {}\n".format(DRs[0])

PT_1
# of detected incidents: 90
Total # of incidents: 125
Detection rate: 72.0
PT_2
# of detected incidents: 87
Total # of incidents: 119
Detection rate: 73.11
PT_4
# of detected incidents: 79
Total # of incidents: 105
Detection rate: 75.24


### AdaBoost

In [159]:
DRs = DR(y_test, y_test_pred_ada, PT_thresholds[0:3], num_dt_segments)
results += "AB DR: {}\n".format(DRs[0])

PT_1
# of detected incidents: 96
Total # of incidents: 125
Detection rate: 76.8
PT_2
# of detected incidents: 94
Total # of incidents: 119
Detection rate: 78.99
PT_4
# of detected incidents: 85
Total # of incidents: 105
Detection rate: 80.95


### MLP

In [160]:
DRs = DR(y_test, y_test_pred_mlp, PT_thresholds[0:3], num_dt_segments)
results += "MLP DR: {}\n".format(DRs[0])

PT_1
# of detected incidents: 102
Total # of incidents: 125
Detection rate: 81.6
PT_2
# of detected incidents: 100
Total # of incidents: 119
Detection rate: 84.03
PT_4
# of detected incidents: 93
Total # of incidents: 105
Detection rate: 88.57


## Mean time to detect (MTTD)

In [161]:
def MTTD(y_test, y_test_pred, PT_thresholds, num_dt_segments):
    MTTDs = []
    for PT_threshold in PT_thresholds:
        h = 0
        sum_ttd = 0
        for i in range(num_dt_segments):
            base_idx = i * 274
            max_base_offset = (i + 1) * 274
            start_idx = base_idx
            end_idx = start_idx
            while start_idx < max_base_offset:
                while start_idx < max_base_offset and y_test[start_idx] == -1:
                    start_idx += 1
                if start_idx == max_base_offset:
                    break
                # an incident happens
                # time span of the incident
                end_idx = start_idx
                while end_idx < max_base_offset and y_test[end_idx] == 1:
                    end_idx += 1

                if end_idx - start_idx >= PT_threshold:
                    # the incident is detected
                    if 1 in y_test_pred[start_idx:end_idx]:
                        h += 1
                        incident_idx = start_idx
                        detection_idx = incident_idx
                        while y_test_pred[detection_idx] == -1:
                            detection_idx += 1
                        sum_ttd += (detection_idx - incident_idx) * 5

                start_idx = end_idx + 1
        MTTDs.append(round(sum_ttd / h, 2))

        print("PT_{}".format(PT_threshold))
        print("Total number of detected incidents: {}".format(h))
        print("Total time of detection lags (min): {}".format(sum_ttd))
        print("Mean time to detect (MTTD): {}".format(MTTDs[-1]))
    return MTTDs

### SVM

In [162]:
MTTDs = MTTD(y_test, y_test_pred, PT_thresholds[0:3], num_dt_segments)
results += "SVM MTTD: {}\n".format(MTTDs[0])

PT_1
Total number of detected incidents: 58
Total time of detection lags (min): 350
Mean time to detect (MTTD): 6.03
PT_2
Total number of detected incidents: 57
Total time of detection lags (min): 350
Mean time to detect (MTTD): 6.14
PT_4
Total number of detected incidents: 50
Total time of detection lags (min): 335
Mean time to detect (MTTD): 6.7


### Decision Tree with Gini Index

In [163]:
MTTDs = MTTD(y_test, y_test_pred_gini, PT_thresholds[0:3], num_dt_segments)
results += "DT MTTD: {}\n".format(MTTDs[0])

PT_1
Total number of detected incidents: 102
Total time of detection lags (min): 765
Mean time to detect (MTTD): 7.5
PT_2
Total number of detected incidents: 99
Total time of detection lags (min): 765
Mean time to detect (MTTD): 7.73
PT_4
Total number of detected incidents: 90
Total time of detection lags (min): 750
Mean time to detect (MTTD): 8.33


### KNN

In [164]:
MTTDs = MTTD(y_test, y_test_pred_knn, PT_thresholds[0:3], num_dt_segments)
results += "KNN MTTD: {}\n".format(MTTDs[0])

PT_1
Total number of detected incidents: 87
Total time of detection lags (min): 825
Mean time to detect (MTTD): 9.48
PT_2
Total number of detected incidents: 86
Total time of detection lags (min): 825
Mean time to detect (MTTD): 9.59
PT_4
Total number of detected incidents: 82
Total time of detection lags (min): 810
Mean time to detect (MTTD): 9.88


### Random Forest

In [165]:
MTTDs = MTTD(y_test, y_test_pred_rdm_forest, PT_thresholds[0:3], num_dt_segments)
results += "RF MTTD: {}\n".format(MTTDs[0])

PT_1
Total number of detected incidents: 67
Total time of detection lags (min): 365
Mean time to detect (MTTD): 5.45
PT_2
Total number of detected incidents: 63
Total time of detection lags (min): 365
Mean time to detect (MTTD): 5.79
PT_4
Total number of detected incidents: 56
Total time of detection lags (min): 360
Mean time to detect (MTTD): 6.43


### Gradient Boosting

In [166]:
MTTDs = MTTD(y_test, y_test_pred_gb, PT_thresholds[0:3], num_dt_segments)
results += "GB MTTD: {}\n".format(MTTDs[0])

PT_1
Total number of detected incidents: 90
Total time of detection lags (min): 700
Mean time to detect (MTTD): 7.78
PT_2
Total number of detected incidents: 87
Total time of detection lags (min): 700
Mean time to detect (MTTD): 8.05
PT_4
Total number of detected incidents: 79
Total time of detection lags (min): 695
Mean time to detect (MTTD): 8.8


### AdaBoost

In [167]:
MTTDs = MTTD(y_test, y_test_pred_ada, PT_thresholds[0:3], num_dt_segments)
results += "AB MTTD: {}\n".format(MTTDs[0])

PT_1
Total number of detected incidents: 96
Total time of detection lags (min): 700
Mean time to detect (MTTD): 7.29
PT_2
Total number of detected incidents: 94
Total time of detection lags (min): 700
Mean time to detect (MTTD): 7.45
PT_4
Total number of detected incidents: 85
Total time of detection lags (min): 680
Mean time to detect (MTTD): 8.0


### MLP

In [168]:
MTTDs = MTTD(y_test, y_test_pred_mlp, PT_thresholds[0:3], num_dt_segments)
results += "MLP MTTD: {}\n".format(MTTDs[0])

PT_1
Total number of detected incidents: 102
Total time of detection lags (min): 535
Mean time to detect (MTTD): 5.25
PT_2
Total number of detected incidents: 100
Total time of detection lags (min): 535
Mean time to detect (MTTD): 5.35
PT_4
Total number of detected incidents: 93
Total time of detection lags (min): 525
Mean time to detect (MTTD): 5.65


## False positive rate

$$ \frac{FP}{N} = \frac{FP}{FP + TN}, $$ where $FP$ = False positive, and $TN$ = True negative.

In [169]:
def FP(y_test, y_test_pred):
    num_false_positives = 0
    num_true_negatives = 0
    total_num_detections = len(y_test_pred)
    for i in range(total_num_detections):
        if y_test_pred[i] == 1 and y_test[i] == -1:
            num_false_positives += 1
        elif y_test_pred[i] == -1 and y_test[i] == -1:
            num_true_negatives += 1
    print("The false positive rate is {}.".format(round(num_false_positives / (num_false_positives + num_true_negatives) * 100, 4)))

In [170]:
FP(y_test, y_test_pred)

The false positive rate is 13.063.


In [171]:
FP(y_test, y_test_pred_gini)

The false positive rate is 32.5635.


In [172]:
FP(y_test, y_test_pred_knn)

The false positive rate is 25.5361.


In [173]:
FP(y_test, y_test_pred_rdm_forest)

The false positive rate is 18.3065.


In [174]:
FP(y_test, y_test_pred_gb)

The false positive rate is 23.2156.


In [175]:
FP(y_test, y_test_pred_mlp)

The false positive rate is 33.4684.


## False alarm rate

In [176]:
def false_alarm_rate(y_test, y_test_pred):
    num_false_alarms = 0
    total_num_detections = len(y_test_pred)
    for i in range(total_num_detections):
        if y_test_pred[i] != y_test[i]:
            num_false_alarms += 1
    far = round(num_false_alarms / total_num_detections * 100, 4)
    print("The false alarm rate is {}.".format(far))
    return far

In [177]:
far = false_alarm_rate(y_test, y_test_pred)
results += "SVM FAR: {}\n".format(far)

The false alarm rate is 13.2805.


In [178]:
far = false_alarm_rate(y_test, y_test_pred_gini)
results += "DT FAR: {}\n".format(far)

The false alarm rate is 32.6588.


In [179]:
far = false_alarm_rate(y_test, y_test_pred_knn)
results += "KNN FAR: {}\n".format(far)

The false alarm rate is 25.6869.


In [180]:
far = false_alarm_rate(y_test, y_test_pred_rdm_forest)
results += "RF FAR: {}\n".format(far)

The false alarm rate is 18.4773.


In [181]:
far = false_alarm_rate(y_test, y_test_pred_gb)
results += "GB FAR: {}\n".format(far)

The false alarm rate is 23.3592.


In [182]:
far = false_alarm_rate(y_test, y_test_pred_ada)
results += "AB FAR: {}\n".format(far)

The false alarm rate is 29.7743.


In [183]:
far = false_alarm_rate(y_test, y_test_pred_mlp)
results += "MLP FAR: {}\n".format(far)

The false alarm rate is 33.5535.


# Print Results

In [184]:
import os

In [185]:
def save_result(output_path, eval_result):
    flag = "w"
    if os.path.isfile(output_path):
        flag = "a"
    
    with open(output_path, flag) as f:
        f.write("{} \n".format(eval_result))
        f.close()

In [186]:
save_result('./res_feature.txt', results)